# **CS 363M Final Project Spring 2025**

## Chenyi Wang, Bhuvan Kannaeganti, Suyog Valsangkar

### **Overview**

For the project in this class, you will participate in a machine learning competition where you’ll apply your ML skills to a real-world dataset. You may work individually or in teams of up to 3 students. 

The dataset for this competition comes from the Austin Animal Center, the largest no-kill animal shelter in the United States. It contains historical records of animals that have entered the shelter, including details such as species, breed, age, intake type, medical condition, and other attributes. Each animal in the dataset has a recorded outcome, which represents what eventually happened to the animal after entering the shelter.

Your goal in this competition is to build a machine learning model that predicts the final outcome of each animal admitted to the shelter, based on its intake characteristics. The possible outcomes are:

*Adopted* – The animal was placed into a new home.<br>
*Return to Owner* – The animal was reclaimed by its original owner.<br>
*Euthanasia* – The animal was humanely euthanized due to medical or behavioral concerns.<br>
*Died* – The animal passed away while in the shelter’s care.<br>
*Transfer* – The animal was moved to another shelter or rescue organization.<br>

By accurately predicting these outcomes, your model can help identify factors that influence an animal's journey through the shelter system and potentially aid in improving adoption and survival rates, shelter policies, or allocation of resources.


## **Code and Analysis Below:**

1. We need to go through the dataset and examine the existing features for patterns and methods we can feature engineer our data to enhance our final predictions. 

In [6]:
import pandas as pd

animal_data = pd.read_csv('train.csv')
animal_data.sample(20)

,Id,Name,Intake Time,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color,Outcome Time,Date of Birth,Outcome Type
46999,A804514,Banjo,09/15/2019 12:53:00 PM,305 West 35Th Street in Austin (TX),Stray,Normal,Cat,Neutered Male,2 years,Siamese Mix,Flame Point,09/16/2019 02:46:00 PM,09/15/2017,Return to Owner
76189,A639421,Wally,04/22/2014 10:57:00 AM,1165 Lott Ave in Austin (TX),Stray,Normal,Dog,Neutered Male,3 years,Pit Bull Mix,Black/White,04/22/2014 01:41:00 PM,10/13/2010,Return to Owner
18843,A879381,A879381,04/25/2023 11:47:00 AM,12313 Caldvis Road in Travis (TX),Stray,Normal,Cat,Intact Female,3 weeks,Domestic Shorthair Mix,Blue Tabby,05/11/2023 12:10:00 PM,04/03/2023,Transfer
7747,A721880,Hamilton,09/16/2016 09:58:00 AM,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,1 year,Labrador Retriever/Boxer,Brown/White,09/22/2016 01:46:00 PM,10/06/2014,Adoption
79121,A726774,Reflection,05/14/2016 04:32:00 PM,Austin (TX),Public Assist,Normal,Cat,Spayed Female,1 year,Domestic Longhair Mix,Black,05/25/2016 04:51:00 PM,05/14/2015,Return to Owner
91090,A711768,Canelo,09/12/2015 09:24:00 AM,Brodie & Hewitt & Fm 1626 in Austin (TX),Stray,Normal,Dog,Intact Male,9 months,Golden Retriever Mix,Red,09/16/2015 05:49:00 PM,11/12/2014,Adoption
73050,A713682,*Francis,10/11/2015 07:57:00 AM,Lynn Street & Canterbury in Austin (TX),Stray,Injured,Cat,Intact Male,1 year,Domestic Shorthair Mix,Brown Tabby,10/12/2015 04:23:00 PM,10/10/2014,Transfer
41280,A793413,Kelly Grace,04/24/2019 04:40:00 PM,1408 Sweet Leaf Lane in Austin (TX),Public Assist,Normal,Cat,Spayed Female,2 years,Domestic Shorthair Mix,Brown Tabby/Black Tabby,04/29/2019 02:59:00 PM,04/24/2017,Return to Owner
69267,A725407,*Nacho,04/28/2016 06:38:00 PM,10302 Old Manchaca Rd in Austin (TX),Stray,Normal,Cat,Intact Male,3 weeks,Domestic Longhair Mix,Blue Tabby,07/06/2016 11:39:00 AM,04/01/2016,Adoption
24035,A742093,*Bailey,01/14/2017 03:18:00 PM,W Rundberg Ln And N Lamar Blvd in Austin (TX),Stray,Normal,Dog,Intact Female,1 year,Australian Shepherd/Border Collie,Black/White,01/19/2017 03:30:00 PM,01/15/2016,Transfer
